<img src="/static/base/images/logo.png?v=641991992878ee24c6f3826e81054a0f" alt="Jupyter Notebook">
<h1 style="text-align: center">Notebook 9 - Blockchain</h1>

<h3>Prerequisites</h3>

- You must have Python 3 installed on your system (<a href="https://www.python.org/downloads/">Download</a>)
- You must have Jupyter installed on your system (<a href="https://jupyter.org/install">Download</a>)
- Some knowledge of Python may be required

<h3>Explanation of Notebook 9</h3>

In this notebook, you will learn what the blockchain is by building your own one using Python, you will use be using Flask to make a web application to replicate a blockchain of your own where you'll be making HTTP requests to create unofficial transactions and send it to the server using chains of blocks, you will also learn how coin mining works as well as verifying blocks to complete these transactions.

<h3>Getting started</h3>

You will be installing two packages: "Flask" and "reqests". <br>
You can do this using pip in Command Prompt (Windows), or the Terminal (MacOS/Linux):<br>
<code>pip install Flask</code><br>
<code>pip install requests</code>

You will also need a HTTP client such as <a href="https://www.getpostman.com/">Postman</a> or <a href="https://curl.haxx.se/download.html">cURL</a> to post and get data from the application.

After doing so, you may proceed with the notebook.

<h3>Before proceeding</h3>

A blockchain is an immutable, sequential chain of records called blocks, and these blocks can contain transactions, filees or any other data and something unique about them is that they are chained together using hashes, which are values created using a hash function.

A hash function is simply a function that takes in an input value creates an output value based on that input value, which means that for any "x" input value, you will always receive the same "y" output value whenever the hash function is run so that every input has a determined output, usually the output is displayed as a hexadecimal number, and so think of this as "encryption" in a way.

<h4>What does a block look like?</h4>

Each block has an index, a timestamp in Unix time, a list of transations as a "proof", and the hash of the previous block.<br>

An example of a single block is shown here:

<code>block = {
    'index': 1,
    'timestamp': 1506057125.900785,
    'transactions': [
        {
            'sender': "8527147fe1f5426f9dd545de4b27ee00",
            'recipient': "a77f5cdfa2934df3954a5c7c7da5df1f",
            'amount': 5,
        }
    ],
    'proof': 324984774000,
    'previous_hash': "2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824"
}</code>

The reason why each block has a hash to the previous block to act as a chain together, meaning that if an earlier block is corrupted by an attacker for instance, all the subsequent blocks after it would contain incorrect hashes.

<h4>What is a proof?</h4>
The term"proof" is short for "Proof of Work", which refers to an algorithm on how new blocks are created or mined on the blockchain, and it serves the purpose of discovering a specific number that solves a specific problem, and that number must be difficult but easy to verify by anyone on the network.

<br>Here's an example; let’s decide that the hash of some integer 'x' multiplied by another 'y' must end in 0 as the last digit of the output, we'll implement this as a "hash decrypting" function using Python with a package known as "sha256" as shown below.

In [ ]:
from hashlib import sha256
x = 5
y = 0  # We don't know what y should be yet

#Continue carrying the calculation out and incrementing y until the last digit it outputs is 0
while sha256(f'{x*y}'.encode()).hexdigest()[-1] != "0":
    y += 1
    
print(f'The solution is y = {y}')

You can see that 'y' is 21 when 'x' is 5, and no matter how many times you run it, you will always get 21, but if you change 'x' to 1000, you'd get even a lower number, and that's why it's really important, because it's difficult to guess as the output always vary.

This is how bitcoin miners are rewarded, they let their computer do the calculation on a much larger scale and in return they receive a bitcoin in a transaction, and it is also what keeps the blockchain running as blocks need to be constantly verified for the transactions within to be successfully sent to the recipient.

<b>Note</b>: <code>f</code> before a quote is used for formatting, hence the variable link to 'y' inside the quotes

<h3>Blockchain object - Building a blockchain</h3>

All the information in the blockchain needs to be stored somewhere, and in this case, it's in an object, and this will include information such as transactions and helper methods to manage the chain of blocks, and so to do this, we will create a class named "Blockchain":

In [ ]:
class Blockchain(object):
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        
    def new_block(self):
        # Creates a new Block and adds it to the chain
        pass
    
    def new_transaction(self):
        # Adds a new transaction to the list of transactions
        pass
    
    @staticmethod
    def hash(block):
        # Hashes a Block
        pass

    @property
    def last_block(self):
        # Returns the last Block in the chain
        pass

For now it is empty, but we will begin with the first two functions; "new_block" and "new_transactions" like shown below.

In [ ]:
import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4

import requests
from flask import Flask, jsonify, request

class Blockchain:
    
    def __init__(self):
        self.current_transactions = [] #List for all current transactions in the chain
        self.chain = [] #The chain of blocks
        self.nodes = set()  #The nodes of the blockchain

        # Create the genesis block
        self.new_block(previous_hash='1', proof=100)
        
    def new_block(self, proof, previous_hash):
        """
        Create a new Block in the Blockchain
        :param proof: The proof given by the Proof of Work algorithm
        :param previous_hash: Hash of previous Block
        :return: New Block
        """

        block = {
            'index': len(self.chain) + 1, #Get the new index of the block
            'timestamp': time(), #Get current time
            'transactions': self.current_transactions, #Get all the transactions
            'proof': proof, #Get the proof
            'previous_hash': previous_hash or self.hash(self.chain[-1]), #Get the previous hash through parameter or index
        }

        # Reset the current list of transactions
        self.current_transactions = []
        
        #Append the block to the chain
        self.chain.append(block)
        
        return block

You can see that it first imports the required packages, and afterwards creates a class named "Blockchain", and in this class there's currently two functions: "new_block" and "__init__".<br>
The "__init__" function is the constructor to the class, meaning that it's what initates it when an object is created for the class, hence why it creates three new variables: "self.current_transactions" (Stores the transactions), "self.chain" (Stores the blockchain), and "self.nodes" (Stores the nodes - Will be explained <a href="#Consensus---Decentralisation-of-our-blockchain">later</a>).

You can see that the constructor calls the "new_block" function with the previous hash as "1" with a proof of "100", this creates a new block that is appended to the chain and it contains all the information in a block such as transactions and the previous hash, but how do you make a transaction? Check below.

In [ ]:
    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: Address of the Sender
        :param recipient: Address of the Recipient
        :param amount: Amount
        :return: The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender, #Set the sender
            'recipient': recipient, #Set the recipient
            'amount': amount, #Set the amount
        })

        return self.last_block['index'] + 1

You can see that the function takes in the sender, recipient and the amount, which gets appended to the transactions list of the block referred to the parameter "self", and then it returns the index of that block, using the last block's attribute for index, which we will complete along with the hash function as shown below.

In [ ]:
    @property #Access the function as an attribute without the '()'
    def last_block(self):
        return self.chain[-1] #The last block in the chain

    @staticmethod #Access the function outside of the object
    def hash(block):
        """
        Creates a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """

        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

While the "last_block" function is explanatory, you can see for the "hash" function that it takes in a block which is encoded into a string that is converted into a sha256 hash right after, and we've already discussed how the hashes work with proof of blocks, but we must include this verification in our system as well, this is shown below.

In [ ]:
    def proof_of_work(self, last_block):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes
         - Where p is the previous proof, and p' is the new proof
         
        :param last_block: <dict> last Block
        :return: <int>
        """

        last_proof = last_block['proof'] #Gets the proof of the last block
        last_hash = self.hash(last_block) #Hashes the last block

        proof = 0
        
        #Increments the proof variable until the following function returns true
        while self.valid_proof(last_proof, proof, last_hash) is False: #Uses the last proof, proof and the last hash to validate the proof
            proof += 1

        return proof

You can see that it uses the previous block from the parameter to grab the proof from its attribute as well as its generated hash as well, and then uses the "valid_proof" function to validate the proof that was grabbed by incrementing the "proof" variable until it returns "True", that function is shown below.

In [ ]:
    @staticmethod
    def valid_proof(last_proof, proof, last_hash):
        """
        Validates the Proof
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :param last_hash: <str> The hash of the Previous Block
        :return: <bool> True if correct, False if not.
        """

        #Attempts to dehash/decrypt the hash
        guess = f'{last_proof}{proof}{last_hash}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000" #Returns True if it contains 0000 at the end

As it gets the last hash, and the proof as well as the last proof, it will try to decrypt the hash using the two proofs to see if the last two digits it'd return is "0000", if not it will return 'True', otherwise 'False', you'll find out that even just adding one extra 0 at the end would take considerably longer to find the solution.

However, since we're almost finished with our class, we will setup Flask so that we can form a single node in our blockchain network using a server, check the cell below.

<h3>Flask application - Use our blockchain as an API</h3>

We will need to interact with our blockchain to make new transactions and more, and for this, we will need to make HTTP requests over the web.

This will involve making three methods:

+ <code>/transactions/new</code> - Create a new transaction to a block
+ <code>/mine</code> - Tell our server to mine a new block.
+ <code>/chain</code> - Return the full Blockchain.

In [ ]:
# Instantiate the Node
app = Flask(__name__)

# Generate a globally unique address for this node
node_identifier = str(uuid4()).replace('-', '')

# Instantiate the Blockchain
blockchain = Blockchain()

We create the flask app, which will create a node and then right after it we make a node identifier for it which contains a unique address using the "uuid4" function, and then we initiate the blockchain.

We still haven't covered what the nodes actually are, but we will get to that soon, but before that, we will need to set the routes for the Flask server, check the cell below.

In [ ]:
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json() #Gets the posted data in JSON format

    # Check that the required fields are in the POST'ed data
    required = ['sender', 'recipient', 'amount']
    if not all(k in values for k in required): #Compares the posted data to see if the three values mentioned above are filled
        return 'Missing values', 400 #Status code

    # Create a new Transaction
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount']) #Uses the values of the posted data
    response = {'message': f'Transaction will be added to Block {index}'} #Response message
    return jsonify(response), 201 #Status code

This creates the route for "/transactions/new" but with the 'POST' method so that it hides the sensitive data when sending to the server, and then it gets the posted data which checks if it is missing by comparing it with the corresponding values which returns "Missing values" if something is missing, but otherwise just a message stating that the transaction will be added to the blockchain along with the block's index.

We can also inspect the full chain by requesting it to see if the block was added to the chain, so we'll need to make a new route for that as well, it is shown below.

In [ ]:
@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200 #Status code

The route is called "/chain" this time, and also with the method 'GET' so that it can be cached and viewed easily, but other than that, it's self explanatory; it gets a response with the chain using the blockchain object created earlier, and gets its length as well and returns it in JSON format.

This means that when the application is complete and the block with the transaction is added, this chain will display that added block at the very end, however, this transaction won't be completed until someone mines that block that was added to the chain, to do this, we must create a new route where it mines the last added block, check the cell below to see how this is done.

In [ ]:
@app.route('/mine', methods=['GET'])
def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    proof = blockchain.proof_of_work(last_block)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0", #Set the sender
        recipient=node_identifier, #Set the recipient
        amount=1, #Set the amount
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'], #Get the index of the block
        'transactions': block['transactions'], #Get the transations of the block
        'proof': block['proof'], #Get the proof of the block
        'previous_hash': block['previous_hash'], #Get previous hash of the block
    }
    return jsonify(response), 200 #Status code

This creates a route for the "/mine" directory using the 'GET' method, meaning that upon visiting /mine, it initiates the last block and the validated proof of that last block, and then in the blockchain it make a new transaction where it denotes that the last block was mined by setting the sender to "0" and sending the bitcoin to the recipient (The "node_identifier" when the server started).

It then gets the previous hash using the last block and creates a new block with that hash and the proof to continue the chain of blocks, and then makes a response that outputs the message, index, transactions, proof, and previous hash of the block once it is mined and returns it at the end in JSON format.

<h3>Consensus - Decentralisation of our blockchain</h3>

We’ve got a basic blockchain that accepts transactions and allows us to mine new blocks, however, the point of blockchains is decentralisation, although the problem with this is if they’re decentralized, we need to ensure that they all reflect the same chain, and this is known as the problem of Consensus; without a Consensus Algorithm, we will only have one node in the network.

<h4>What exactly is a node?</h4>

A node is simply any electronic device on a blockchain network ranging from a computer, phone, and even a prrinter. You've already heard of blocks, and these blocks contains data, but where are these blocks stored? They are stored in the nodes. These nodes form the infrastructure of the blockchain, and that's why it's decentralised, because every node/device keeps a full transaction history of the blockchain meaning that it's very unlikely to be compromised and have its data changed as then all of the nodes must have their data changes as well one by one because not only do each has a copy of these transactions but also exchange it with other nodes, and so it is very hard for a hacker to breach a blockchain.

We learnt that by creating the Flask application we essentially form a node, however, this doesn't mean that it's registered on the blockchain, and so we must create a new function for this like below.

In [ ]:
    def register_node(self, address):
        """
        Add a new node to the list of nodes
        :param address: Address of node. Eg. 'http://192.168.0.5:5000'
        """

        parsed_url = urlparse(address) #Parse the address as a URL
        if parsed_url.netloc: #192.168.0.5
            self.nodes.add(parsed_url.netloc) #Add the first level domain
        elif parsed_url.path: #192.168.0.5:5000
            self.nodes.add(parsed_url.path) #Add the path
        else:
            raise ValueError('Invalid URL')

This takes in the given address as the parameter which is parsed as a URL, and checks if it contains the first level domain first (E.g 192.168.0.5), which if it is, it adds it to the set of nodes made in the blockchain earlier on, and it's the same if it's a path (E.g 192.168.0.5:5000) which contains the port. However, if it's neither, it will raise the error as an invalid URL.

We can now validate chain to ensure that all the chains have a valid proof and that the previous hash of each block matches the hash on the previous block, this is how we ensure that the chain is up to date and that no other chain differs, this is shown on the below cell.

In [ ]:
     def valid_chain(self, chain):
        """
        Determine if a given blockchain is valid
        :param chain: A blockchain
        :return: True if valid, False if not
        """

        last_block = chain[0] #Gets first block as the last block
        current_index = 1 #Set the current index

        #Validate each block in the chain until it reaches the end
        while current_index < len(chain):
            block = chain[current_index] #Get the current block's index
            print(f'{last_block}') #Print the last block
            print(f'{block}') #Print the current block
            print("\n-----------\n")
            # Check that the hash of the block is correct
            last_block_hash = self.hash(last_block) #Get the hash of the previous block
            if block['previous_hash'] != last_block_hash: #If the previous block's hash doesn't match current block's previous hash
                return False

            # Check that the Proof of Work is correct
            if not self.valid_proof(last_block['proof'], block['proof'], last_block_hash): #If it returns False when using the last block's proof and hash and current block's proof
                return False

            last_block = block #Reset the last block to the current block
            current_index += 1 #Increment the index

        return True

You can see that it takes the first block and from there on until the very end of the chain, it uses the the both of their hashes as well as their proofs to validate the chain, meaning that if either of them return 'False', it means that the chain is invalid, but otherwise when the loop is finished, it will naturally return 'True'.

Using the "valid_chain" method above, we must check if there are conflicts in the chain by verifying chain of all the nodes in the blockchain, if there are any conflicts, we must replace the chain with the longest one in the network that is deemed "valid", this is known as the "consensus algorithm", and it is shown below.

In [ ]:
    def resolve_conflicts(self):
        """
        This is our consensus algorithm, it resolves conflicts
        by replacing our chain with the longest one in the network.
        :return: True if our chain was replaced, False if not
        """

        neighbours = self.nodes #Gets the current nodes
        new_chain = None #Makes a new chain

        # We're only looking for chains longer than ours
        max_length = len(self.chain) #Gets the length of the chain

        # Grab and verify the chains from all the nodes in our network
        for node in neighbours:
            response = requests.get(f'http://{node}/chain')  #Get chain data in JSON format using the full URL

            if response.status_code == 200: #If no values are missing
                length = response.json()['length'] #Get the length of the chain data for the current node
                chain = response.json()['chain'] #Gets the chain of the chain data for the current node

                # Check if the length is longer and the chain is valid
                if length > max_length and self.valid_chain(chain):
                    max_length = length #Set the current length as the maximum length
                    new_chain = chain #Set the current chain as the

        # Replace our chain if we discovered a new, valid chain longer than ours
        if new_chain:
            self.chain = new_chain #Get the last chain that was set
            return True

        #If there is no valid chain longer than ours
        return False

You can see that this gets all the nodes from the blockchain and creates a new chain of blocks where for each node that was previously gained, if the node's chain is valid (using the "valid_chain" function) and has a higher length than the current chain, it replaces it, making that node's chain the main chain.

<h3>Registering and resolving nodes in the Flask application</h3>

Now that we have the functions to register a node as well as resolve any conflicts in the chain, we have to set up new routes in the application/server for the blockchain to be complete, we'll start with registering a node.

In [ ]:
@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json() #Gets the posted data in JSON format

    nodes = values.get('nodes') #Gets the nodes of the data
    
    #Ensure there's a list of nodes in the data
    if nodes is None:
        return "Error: Please supply a valid list of nodes", 400

    for node in nodes:
        blockchain.register_node(node) #Register each node to the blockchain using the given list

    response = {
        'message': 'New nodes have been added',
        'total_nodes': list(blockchain.nodes), #Get the list of the new nodes as a list
    }
    return jsonify(response), 201 #Status code

This gets the list of nodes using the posted data using the 'POST' method and ensures that it contains nodes, and if it does, it registers each of those nodes into the blockchain and returns a message with the overall nodes in the blockchains as a list in JSON format.

Now we need to set the route to verify and resolve any conflicts, this is shown below.

In [ ]:
@app.route('/nodes/resolve', methods=['GET'])
def consensus():
    replaced = blockchain.resolve_conflicts() #Returns if any conflicts were resolved or not

    if replaced: #If the chain was replaced with a newer one
        response = {
            'message': 'Our chain was replaced',
            'new_chain': blockchain.chain
        }
    else: #If the chain stayed the same
        response = {
            'message': 'Our chain is authoritative',
            'chain': blockchain.chain
        }

    return jsonify(response), 200 #Status code

This route is created with the 'GET' method, it verifies all the chains by iterating through all nodes in the blockchain and replaces the main chain if needed (All using the "resolve.conflicts" function), and at the end it displays the new or the old chain along with whether the chain was replaced or not in JSON format.

<h3>Running the Flask application</h3>

Now that we finished completing our very own blockchain, the entire thing is completed by the following cell.

In [ ]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

Running the cell above will most likely result in errors when making the HTTP requests, this is because the functions of the blockchain class needs to be together when it's run, therefore the full code is provided below.

<h3>Full code</h3>

In [ ]:
import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4

import requests
from flask import Flask, jsonify, request

class Blockchain:
    
    def __init__(self):
        self.current_transactions = [] #List for all current transactions in the chain
        self.chain = [] #The chain of blocks
        self.nodes = set()  #The nodes of the blockchain

        # Create the genesis block
        self.new_block(previous_hash='1', proof=100)
        
    def new_block(self, proof, previous_hash):
        """
        Create a new Block in the Blockchain
        :param proof: The proof given by the Proof of Work algorithm
        :param previous_hash: Hash of previous Block
        :return: New Block
        """

        block = {
            'index': len(self.chain) + 1, #Get the new index of the block
            'timestamp': time(), #Get current time
            'transactions': self.current_transactions, #Get all the transactions
            'proof': proof, #Get the proof
            'previous_hash': previous_hash or self.hash(self.chain[-1]), #Get the previous hash through parameter or index
        }

        # Reset the current list of transactions
        self.current_transactions = []
        
        #Append the block to the chain
        self.chain.append(block)
        
        return block
    
    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: Address of the Sender
        :param recipient: Address of the Recipient
        :param amount: Amount
        :return: The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender, #Set the sender
            'recipient': recipient, #Set the recipient
            'amount': amount, #Set the amount
        })

        return self.last_block['index'] + 1

    @property #Access the function as an attribute without the '()'
    def last_block(self):
        return self.chain[-1] #The last block in the chain

    @staticmethod #Access the function outside of the object
    def hash(block):
        """
        Creates a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """

        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def proof_of_work(self, last_block):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes
         - Where p is the previous proof, and p' is the new proof

        :param last_block: <dict> last Block
        :return: <int>
        """

        last_proof = last_block['proof'] #Gets the proof of the last block
        last_hash = self.hash(last_block) #Hashes the last block

        proof = 0

        #Increments the proof variable until the following function returns true
        while self.valid_proof(last_proof, proof, last_hash) is False: #Uses the last proof, proof and the last hash to validate the proof
            proof += 1

        return proof
    
    @staticmethod
    def valid_proof(last_proof, proof, last_hash):
        """
        Validates the Proof
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :param last_hash: <str> The hash of the Previous Block
        :return: <bool> True if correct, False if not.
        """

        #Attempts to dehash/decrypt the hash
        guess = f'{last_proof}{proof}{last_hash}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000" #Returns True if it contains 0000 at the end
    
    def register_node(self, address):
        """
        Add a new node to the list of nodes
        :param address: Address of node. Eg. 'http://192.168.0.5:5000'
        """

        parsed_url = urlparse(address) #Parse the address as a URL
        if parsed_url.netloc: #192.168.0.5
            self.nodes.add(parsed_url.netloc) #Add the first level domain
        elif parsed_url.path: #192.168.0.5:5000
            self.nodes.add(parsed_url.path) #Add the path
        else:
            raise ValueError('Invalid URL')
        
    def valid_chain(self, chain):
        """
        Determine if a given blockchain is valid
        :param chain: A blockchain
        :return: True if valid, False if not
        """

        last_block = chain[0] #Gets first block as the last block
        current_index = 1 #Set the current index

        #Validate each block in the chain until it reaches the end
        while current_index < len(chain):
            block = chain[current_index] #Get the current block's index
            print(f'{last_block}') #Print the last block
            print(f'{block}') #Print the current block
            print("\n-----------\n")
            # Check that the hash of the block is correct
            last_block_hash = self.hash(last_block) #Get the hash of the previous block
            if block['previous_hash'] != last_block_hash: #If the previous block's hash doesn't match current block's previous hash
                return False

            # Check that the Proof of Work is correct
            if not self.valid_proof(last_block['proof'], block['proof'], last_block_hash): #If it returns False when using the last block's proof and hash and current block's proof
                return False

            last_block = block #Reset the last block to the current block
            current_index += 1 #Increment the index

        return True
    
    def resolve_conflicts(self):
        """
        This is our consensus algorithm, it resolves conflicts
        by replacing our chain with the longest one in the network.
        :return: True if our chain was replaced, False if not
        """

        neighbours = self.nodes #Gets the current nodes
        new_chain = None #Makes a new chain

        # We're only looking for chains longer than ours
        max_length = len(self.chain) #Gets the length of the chain

        # Grab and verify the chains from all the nodes in our network
        for node in neighbours:
            response = requests.get(f'http://{node}/chain')  #Get chain data in JSON format using the full URL

            if response.status_code == 200: #If no values are missing
                length = response.json()['length'] #Get the length of the chain data for the current node
                chain = response.json()['chain'] #Gets the chain of the chain data for the current node

                # Check if the length is longer and the chain is valid
                if length > max_length and self.valid_chain(chain):
                    max_length = length #Set the current length as the maximum length
                    new_chain = chain #Set the current chain as the

        # Replace our chain if we discovered a new, valid chain longer than ours
        if new_chain:
            self.chain = new_chain #Get the last chain that was set
            return True

        #If there is no valid chain longer than ours
        return False
    
# Instantiate the Node
app = Flask(__name__)

# Generate a globally unique address for this node
node_identifier = str(uuid4()).replace('-', '')

# Instantiate the Blockchain
blockchain = Blockchain()

@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json() #Gets the posted data in JSON format

    # Check that the required fields are in the POST'ed data
    required = ['sender', 'recipient', 'amount']
    if not all(k in values for k in required): #Compares the posted data to see if the three values mentioned above are filled
        return 'Missing values', 400 #Status code

    # Create a new Transaction
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount']) #Uses the values of the posted data
    response = {'message': f'Transaction will be added to Block {index}'} #Response message
    return jsonify(response), 201 #Status code

@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200 #Status code
    
@app.route('/mine', methods=['GET'])
def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    proof = blockchain.proof_of_work(last_block)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0", #Set the sender
        recipient=node_identifier, #Set the recipient
        amount=1, #Set the amount
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'], #Get the index of the block
        'transactions': block['transactions'], #Get the transations of the block
        'proof': block['proof'], #Get the proof of the block
        'previous_hash': block['previous_hash'], #Get previous hash of the block
    }
    return jsonify(response), 200 #Status code

@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json() #Gets the posted data in JSON format

    nodes = values.get('nodes') #Gets the nodes of the data
    
    #Ensure there's a list of nodes in the data
    if nodes is None:
        return "Error: Please supply a valid list of nodes", 400

    for node in nodes:
        blockchain.register_node(node) #Register each node to the blockchain using the given list

    response = {
        'message': 'New nodes have been added',
        'total_nodes': list(blockchain.nodes), #Get the list of the new nodes as a list
    }
    return jsonify(response), 201 #Status code

@app.route('/nodes/resolve', methods=['GET'])
def consensus():
    replaced = blockchain.resolve_conflicts() #Returns if any conflicts were resolved or not

    if replaced: #If the chain was replaced with a newer one
        response = {
            'message': 'Our chain was replaced',
            'new_chain': blockchain.chain
        }
    else: #If the chain stayed the same
        response = {
            'message': 'Our chain is authoritative',
            'chain': blockchain.chain
        }

    return jsonify(response), 200 #Status code

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

<h3>Running and using the blockchain</h3>

Run the full code cell above and copy and paste the URL or type "127:0.0.1:5000" in a new tab and go to it, you'll notice something different; there's no interface or any way to navigate through this web application, it simply says "URL not found" or something similar when you visit the page, and this is because we need a HTTP client as mentioned earlier.

For this example, we will use Postman as it's more user-friendly, so after you install it and run the application above, open Postman.<br>

When you first open Postman after signing up or skipping it, you will be presented the following screen:

<img src="https://i.imgur.com/6mhnwey.png">

+ Press the "<b>Request</b>" button to begin.

<h4>Mining a block</h4>

+ First, we will mine a block. To do this, press the "+" button on the top left like shown:

<img src="https://i.imgur.com/ssS6yom.png">

+ Type in the field next to the selected method the following URL: "localhost:5000/mine" and then press the "<b>Send</b>" button to make a 'GET' request.
+ <b>cuRL alternative</b> (Type in Command Prompt (Windows), or the Terminal (MacOS/Linux)):<br>
<code>curl 127.0.0.1:5000/mine</code>

<img src="https://i.imgur.com/LW6cFMi.png">

You can see that it returns a body with the following:

<code>{
    "index": 2,
    "message": "New Block Forged",
    "previous_hash": "579c0bdf9444a2a9eb8a09e9fe5cf896e278f0a9391b9dcd90e438d7b2cfae56",
    "proof": 563556,
    "transactions": [
        {
            "amount": 1,
            "recipient": "fba4e0a2e12a44c298bc20cee40b2792",
            "sender": "0"
        }
    ]
}</code>

<br>Notice how the "index" is set to 2, this is because when the blockchain server started, it created a genesis block; the very first block in the blockchain. The hash of that block is included in this block as "previous_hash". You can also see the list of transactions, which in this case is just one transaction with the recipient, amount and the sender (Set to 0 because the block was mined).

<h4>Making a transaction</h4>

+ Second, we will make a transaction, set the method to 'POST' and set the request URL to "localhost:5000/transactions/new" this time:

<img src="https://i.imgur.com/rTN3qzn.png">

+ Select the "Body" tab and select the "raw" button and change the type from "Text" to "JSON (application/json)" and type the following:

<code>{
 "sender": "d4ee26eee15148ee92c6cd394edd974e",
 "recipient": "someone-other-address",
 "amount": 5
}</code>

+ Then press the "<b>Send</b>" button <i>twice</i>.
+ <b>cuRL alternative</b> (Type in Command Prompt (Windows), or the Terminal (MacOS/Linux)):<br>
<code>curl -H "Content-Type: application/json" -X POST -d "{""sender"":""d4ee26eee15148ee92c6cd394edd974e"",""recipient"":""someone-other-address"",""amount"":""5""}" http://localhost:5000/transactions/new</code>

<img src="https://i.imgur.com/SoILw8b.png">

Notice how it's on block 3, this is because the previous block (Block 2) was already mined, so a new block is created. <br>
So now, instead of a new transaction, if we mine once more, you should see the following:

<img src="https://i.imgur.com/pAnZbjv.png">

The output is the following:

<code>{
    "index": 3,
    "message": "New Block Forged",
    "previous_hash": "415649965054ab123f3c26774fcd5174e12e6ba18c0ef06d12ee1d7615c15895",
    "proof": 76645,
    "transactions": [
        {
            "amount": 5,
            "recipient": "someone-other-address",
            "sender": "d4ee26eee15148ee92c6cd394edd974e"
        },
        {
            "amount": 5,
            "recipient": "someone-other-address",
            "sender": "d4ee26eee15148ee92c6cd394edd974e"
        },
        {
            "amount": 1,
            "recipient": "deecd51ac0184ed69887f0c6c934fa9d",
            "sender": "0"
        }
    ]
}</code>

Notice how there are three transactions, two for the two transactions that we made, and an extra transaction with the amount set to 1, and as said previously, having the sender's value set to 0 means that the user has mined a block, which is this block in this case. This should hopefully give an idea of how storing data in blocks work.

<h4>Registering a node</h4>

+ Third, we will register a node, to do this, we will set the request URL to "http://localhost:5000/nodes/register", make sure that the method is set to 'POST', and then select the "Body" tab and select the "raw" button and change the type from "Text" to "JSON (application/json)" and type the following:

<code>{
    "nodes": ["http://127.0.0.1:5001"]
}</code>

+ <b>cuRL alternative</b> (Type in Command Prompt (Windows), or the Terminal (MacOS/Linux)):<br>
<code>curl -H "Content-Type: application/json" -X POST -d "{""nodes"":[""http://127.0.0.1:5001""]}" http://localhost:5000/nodes/register</code>

<img src="https://i.imgur.com/f180S6A.png">

This outputs the following:

+ Then press the "<b>Send</b>" button.

<code>{
    "message": "New nodes have been added",
    "total_nodes": [
        "127.0.0.1:5001"
    ]
}</code>

The "total_nodes" is a list that stores all the nodes in the blockchain, which in this case is only "127.0.0.1:5001" as it's the only node, the reason why we used the port "5001" instead of "5000" is to show how other nodes in <i>our</i> blockchain would differ, so for our node, we'd add "127.0.0.1:5000", but you can run another machine with this blockchain with the port "5001" to differ from the first machine.

<h4>Resolving conflicts in the chain</h4>

+ Fourth, we can resolve any found conflicts in the chain. This time, it's different, because you'd have to run two machines with our blockchain on each of those two machines, but because it's not possible to show that in this notebook, an example is shown below (<a href="https://www.experfy.com/blog/learn-blockchains-by-building-one">Source</a>) where the method is set to "GET" and a request is sent to "http://localhost:5000/nodes/resolve".
+ <b>cuRL alternative</b> (Type in Command Prompt (Windows), or the Terminal (MacOS/Linux)):<br>
<code>curl http://localhost:5000/nodes/resolve</code>

<img src="https://cdn-images-1.medium.com/max/800/1*SGO5MWVf7GguIxfz6S8NVw.png">

Incase it is harder to read, that returns a message stating whether or not the chain was replaced along with the current/new chain with all its details of their blocks.

<h3>The end</h3>

This concludes the end of notebook 9, you should now have a better understanding of what the blockchain is as well as how it works as you've just built one, and it should also give you an idea of how to use HTTP requests in web applications for testing. 

Do note that this blockchain doesn't entirely reflect on a professional implementation of a blockchain as there's many more things involved such as security.

<h3>Bibliography</h3>

+ <a href="http://localhost:8888/lab/workspaces/auto-g#Python---Further-exploration">Learn Blockchains by Building One</a> by Daniel van Flymen, 2017 - Retrieved 21st of July, 2019, from <a href="https://hackernoon.com/learn-blockchains-by-building-one-117428612f46">https://hackernoon.com/learn-blockchains-by-building-one-117428612f46</a> and <a href="https://www.experfy.com/blog/learn-blockchains-by-building-one">https://www.experfy.com/blog/learn-blockchains-by-building-one</a>.